<a href="https://colab.research.google.com/github/aarnavg54/Deep-Learning-Radiomic-Stability/blob/main/DeepLabV3%2B_with_resnext101_32x8d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installing neccessary libraries
!pip install -U segmentation-models-pytorch
!pip install -U git+https://github.com/qubvel-org/segmentation_models.pytorch

In [ ]:
import torch
from torch.utils.data import Dataset

# This function creates a pytorch dataset
class HistologyDataset(Dataset):
    def __init__(self, images, masks):
        self.images = images
        self.masks = masks

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx].astype("float32")
        mask = self.masks[idx].astype("float32")
        image = torch.from_numpy(image).permute(2, 0, 1)
        mask = torch.from_numpy(mask).permute(2, 0, 1)
        return image, mask


In [ ]:
# Mounting Google Colab
from google.colab import drive
drive.mount('/content/drive')

# Our preprocessed data
import numpy as np
X_train = np.load('/content/drive/MyDrive/X_train_ultrasound_images_256_2.npy')
y_train = np.load('/content/drive/MyDrive/y_train_ultrasound_images_256_2.npy')
X_val = np.load('/content/drive/MyDrive/X_val_ultrasound_images_256_2.npy')
y_val = np.load('/content/drive/MyDrive/y_val_ultrasound_images_256_2.npy')
X_test = np.load('/content/drive/MyDrive/X_test_ultrasound_images_256_2.npy')
y_test = np.load('/content/drive/MyDrive/y_test_ultrasound_images_256_2.npy')

train_dataset = HistologyDataset(X_train, y_train)
val_dataset = HistologyDataset(X_val, y_val)
test_dataset = HistologyDataset(X_test, y_test)

In [ ]:
# Sanity check
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)
print(X_test.shape)
print(y_test.shape)

In [ ]:
# Importing our model from Qubvel's Segmentation Models GitHub repo
import segmentation_models_pytorch as smp

model = smp.DeepLabV3Plus(
    encoder_name="resnext101_32x8d",
    encoder_weights="imagenet",
    in_channels=3,
    classes=1,
    activation=None
)

In [ ]:
# Dice loss
from segmentation_models_pytorch.losses import DiceLoss
import torch.optim as optim

dice_loss = DiceLoss(mode='binary', from_logits=True, smooth=1e-5)
optimizer = optim.Adam(model.parameters(), lr=1e-4)

# Setting up the training loop
from torch.utils.data import DataLoader
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [ ]:
# Loading our training/ validation/ testing data
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)  # UPDATED
val_loader = DataLoader(val_dataset, batch_size=4)
test_loader = DataLoader(test_dataset)

In [ ]:
# Initializing the number of epochs
num_epochs = 500
# Early stopping settings
patience = 10
best_val_loss = float('inf')
epochs_no_improve = 0

checkpoint_dir = "./checkpoints"
os.makedirs(checkpoint_dir, exist_ok=True)
checkpoint_path = os.path.join(checkpoint_dir, "best_model.pth")

In [ ]:
# Looping through all 500 epochs
for epoch in range(1, num_epochs + 1):
    model.train()
    train_losses = []
    for images, masks in train_loader:
        images, masks = images.to(device), masks.to(device)

        optimizer.zero_grad()
        outputs = model(images)

        loss = dice_loss(outputs, masks)
        loss.backward()
        optimizer.step()

        train_losses.append(loss.item())

    avg_train_loss = sum(train_losses) / len(train_losses)

    model.eval()
    val_losses = []
    with torch.no_grad():
        for images, masks in val_loader:
            images, masks = images.to(device), masks.to(device)
            outputs = model(images)

            loss = dice_loss(outputs, masks)
            val_losses.append(loss.item())

    avg_val_loss = sum(val_losses) / len(val_losses)
    # Printing as training/ validation loss as the model trains

    print(f"Epoch {epoch:03d}: Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f}")

    # Early stopping conditions
    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        epochs_no_improve = 0
        torch.save(model.state_dict(), checkpoint_path)
        print(f"Best model saved with Val Loss: {best_val_loss:.4f}")
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= patience:
            print("Early stopping triggered.")
            break

    # If the model's validation loss doesn't improve after 10 epochs, the model will automatically trigger early stopping

In [ ]:
from scipy.spatial.distance import directed_hausdorff

def hausdorff_distance(pred, target):
    pred = pred.squeeze().cpu().numpy()
    target = target.squeeze().cpu().numpy()

    # Get coordinates of boundary pixels
    pred_coords = np.argwhere(pred > 0.5)
    target_coords = np.argwhere(target > 0.5)

    # Handle empty masks
    if len(pred_coords) == 0 or len(target_coords) == 0:
        return np.nan  # Return NaN if either mask is empty

    # Compute both directions
    hd1 = directed_hausdorff(pred_coords, target_coords)[0]
    hd2 = directed_hausdorff(target_coords, pred_coords)[0]

    return max(hd1, hd2)

In [ ]:
def calculate_metrics(preds, targets, smooth=1e-6):
    # Flatten tensors and calculate TP, FP, FN, TN
    preds_flat = preds.flatten(1)
    targets_flat = targets.flatten(1)

    tp = (preds_flat * targets_flat).sum(1)
    fp = (preds_flat * (1 - targets_flat)).sum(1)
    fn = ((1 - preds_flat) * targets_flat).sum(1)
    tn = ((1 - preds_flat) * (1 - targets_flat)).sum(1)

    # Standard metrics
    metrics = {
        'Dice': ((2 * tp + smooth) / (tp + fp + tp + fn + smooth)).mean().item(),
        'IoU': ((tp + smooth) / (tp + fp + fn + smooth)).mean().item(),
        'Precision': ((tp + smooth) / (tp + fp + smooth)).mean().item(),
        'Recall': ((tp + smooth) / (tp + fn + smooth)).mean().item(),
        'Specificity': ((tn + smooth) / (tn + fp + smooth)).mean().item(),
        'Accuracy': ((tp + tn + smooth) / (tp + tn + fp + fn + smooth)).mean().item(),
    }

    # Hausdorff Distance (handle batch)
    hd_values = []
    for p, t in zip(preds, targets):
        hd = hausdorff_distance(p, t)
        if not np.isnan(hd):
            hd_values.append(hd)
    metrics['HD95'] = np.percentile(hd_values, 95) if hd_values else np.nan

    # Tumor size error
    pred_area = preds_flat.sum(1)
    target_area = targets_flat.sum(1)
    metrics['Size_Error'] = ((pred_area - target_area).abs() / (target_area + smooth)).mean().item()

    return metrics

In [ ]:
model.eval()
all_metrics = []

with torch.no_grad():
    for images, masks in test_loader:
        images, masks = images.to(device), masks.to(device)
        outputs = model(images)
        preds = (torch.sigmoid(outputs) > 0.5).float()
        metrics = calculate_metrics(preds, masks)
        all_metrics.append(metrics)

# Aggregate results (ignoring NaN values)
final_metrics = {
    k: np.nanmean([m[k] for m in all_metrics if not np.isnan(m[k])])
    for k in all_metrics[0].keys()
}


In [ ]:
print("Final Metrics:")
for metric, value in final_metrics.items():
    print(f"{metric}: {value:.4f}")

In [ ]:
import numpy as np
from tqdm import tqdm

def bootstrap_ci(metric_values, n_bootstraps=1000, ci=95, seed=42):
    """Calculate bootstrap confidence intervals"""
    np.random.seed(seed)
    bootstrapped_means = []

    for _ in tqdm(range(n_bootstraps)):
        # Resample with replacement
        resampled = np.random.choice(metric_values, size=len(metric_values), replace=True)
        bootstrapped_means.append(np.nanmean(resampled))

    lower = np.percentile(bootstrapped_means, (100 - ci) / 2)
    upper = np.percentile(bootstrapped_means, 100 - (100 - ci) / 2)
    return lower, upper

# Usage
dice_scores = [m['Dice'] for m in all_metrics]
dice_ci = bootstrap_ci(dice_scores, ci=95)
print(f"Dice: {np.mean(dice_scores):.3f} ({dice_ci[0]:.3f}-{dice_ci[1]:.3f})")

In [ ]:
from scipy import stats

def parametric_ci(metric_values, ci=95):
    """Calculate parametric CI assuming normal distribution"""
    mean = np.nanmean(metric_values)
    sem = stats.sem(metric_values, nan_policy='omit')  # Standard error
    ci_val = sem * stats.t.ppf((1 + ci/100) / 2, len(metric_values)-1)
    return mean - ci_val, mean + ci_val

# Usage
iou_scores = [m['IoU'] for m in all_metrics]
iou_ci = parametric_ci(iou_scores, ci=95)
print(f"IoU: {np.mean(iou_scores):.3f} ({iou_ci[0]:.3f}-{iou_ci[1]:.3f})")


In [ ]:
def evaluate_with_ci(model, test_loader, device, n_bootstraps=1000):
    model.eval()
    all_metrics = []

    with torch.no_grad():
        for images, masks in test_loader:
            images, masks = images.to(device), masks.to(device)
            outputs = model(images)
            preds = (torch.sigmoid(outputs) > 0.5).float()
            metrics = calculate_metrics(preds, masks)
            all_metrics.append(metrics)

    # Aggregate metrics
    metric_names = all_metrics[0].keys()
    results = {}

    for name in metric_names:
        values = [m[name] for m in all_metrics]

        # Use bootstrapping for Dice/IoU/HD95
        if name in ['Dice', 'IoU', 'HD95']:
            ci = bootstrap_ci(values, n_bootstraps=n_bootstraps)
        else:
            ci = parametric_ci(values)

        results[name] = {
            'mean': np.nanmean(values),
            'ci_lower': ci[0],
            'ci_upper': ci[1],
            'ci_method': 'bootstrap' if name in ['Dice', 'IoU', 'HD95'] else 'parametric'
        }

    return results


In [ ]:
results = evaluate_with_ci(
    model=model,
    test_loader=test_loader,
    device=device,
    n_bootstraps=1000  # Reduce to 200-500 for faster testing
)

# Print formatted results
for metric, vals in results.items():
    print(f"{metric}: {vals['mean']:.4f} ({vals['ci_lower']:.4f}-{vals['ci_upper']:.4f}) [CI method: {vals['ci_method']}]")

In [ ]:
checkpoint_path = '/content/drive/MyDrive/U-Net_0.9259.pth'

In [ ]:
torch.save(model.state_dict(), checkpoint_path)
print(f"Model saved to {checkpoint_path}")